# T1555 - Credentials from Password Stores
Adversaries may search for common password storage locations to obtain user credentials. Passwords are stored in several places on a system, depending on the operating system or application holding the credentials. There are also specific applications that store passwords to make it easier for users manage and maintain. Once credentials are obtained, they can be used to perform lateral movement and access restricted information.

## Atomic Tests:
Currently, no tests are available for this technique.

## Detection
Monitor system calls, file read events, and processes for suspicious activity that could indicate searching for a password  or other activity related to performing keyword searches (e.g. password, pwd, login, store, secure, credentials, etc.) in process memory for credentials. File read events should be monitored surrounding known password storage applications.

# Shield Active Defense
## Decoy Credentials 
 Create user credentials that are used for active defense purposes. 

 Seed a target system with credentials (such as username/password, browser tokens, and other forms of authentication data) for the purpose of engagement. Decoy credentials can be planted in many locations and leveraged in a variety of ways.
### Opportunity
In order to prolong an adversary engagement operation or enable detections, there is an opportunity to introduce credentials to an adversary that you want them to collect and use.
### Use Case
A defender can plant decoy credentials across an array of locations to increase the chances of an adversary finding and using them.
### Procedures
Create user credentials for a decoy account, such as 'User ABC'.  Store those credentials in the browser and other places on the system to see if an adversary attempts to harvest them.